In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/samples/notebooks/prediction/SDK_Triton_PyTorch_Local_Prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial demonstrates how to use Vertex AI SDK to locally test [NVIDIA Triton inference server](https://developer.nvidia.com/nvidia-triton-inference-server) to serve a PyTorch model and deploy it to Vertex AI Predictions. This is currently an **experimental** feature and is not yet officially supported by the Vertex AI SDK. In this tutorial, we'll be installing the Vertex AI SDK from an experimental branch on github. 



### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small dataset that is popular for trying out machine learning techniques. Each instance has four numerical features, which are different measurements of a flower, and a target label that
marks it as one of three types of iris: Iris setosa, Iris versicolour, or Iris virginica.

This tutorial uses [the Iris dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/iris).

### Objective

The goal is to:
- Train a model that uses a flower's measurements as input to predict what type of iris it is with PyTorch.
- Save the model.
- Test the NVIDIA Triton inference server locally.
- Upload and deploy Triton inference server as custom container to Vertex Prediction.

This tutorial focuses more on deploying this model with Vertex AI than on
the design of the model itself.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* Docker
* Git
* Google Cloud SDK (gcloud)
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as NumPy, Scikit-learn, FastAPI, Uvicorn, and joblib. Use the latest major GA version of each package.

In [ ]:
%%writefile requirements.txt
pandas
torch==1.11.0
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction] @ git+https://github.com/googleapis/python-aiplatform.git@custom-prediction-routine

**The model you deploy will have a different set of dependencies pre-installed than your notebook environment has. You should not assume that because things work in the notebook, they will work in the model. Instead, we will be very explicit about the dependencies for the model by listing them in requirements.txt and then use `pip install` to install the exact same dependencies in the notebook. Please note, of course, that there is a chance that we miss a dependency in requirements.txt that already exists in the notebook. If that's the case, things will run in the notebook, but not in the model. To guard against that, we will test the model locally before deploying to the cloud.**

In [ ]:
# Install the same dependencies used in the serving container in the notebook
# environment.
%pip install -U --user -r requirements.txt

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` or `%` as shell commands, and it interpolates Python variables with `$` or `{}` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

print("Project ID:", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[PROJ_ID]"  # @param {type:"string"}

### Configure project and resource names

In [ ]:
REGION = "us-central1"  # @param {type:"string"}
MODEL_ARTIFACT_DIR = "triton-pytorch"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction-sdk"  # @param {type:"string"}
IMAGE = "triton-pytorch"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "triton-pytorch"  # @param {type:"string"}

`REGION` - Used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability). You may
not use a Multi-Regional Storage bucket for prediction with Vertex AI.

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that will be pushed.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI Model resource.

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

To update your model artifacts without re-building the container, you must upload your model
artifacts and any custom code to Cloud Storage.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets and start with `gs://`.

In [ ]:
BUCKET_NAME = "gs://[BUCKET_NAME]"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Set up directories

Decide the directory to put your model artifacts.

In [ ]:
MODEL_ARTIFACTS_DIRECTORY = "model_artifacts"  # @param {type:"string"}

In [ ]:
%mkdir $MODEL_ARTIFACTS_DIRECTORY

## Training a PyTorch model

### Download iris data
In this example, we want to build a classifier for the simple [iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). So first, we download the data csv file locally.

In [ ]:
DATA_DIR = "data_iris"

%mkdir $DATA_DIR

LOCAL_DATA_FILE = f"{DATA_DIR}/iris.csv"

In [ ]:
from urllib.request import urlretrieve

urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", LOCAL_DATA_FILE)

### Build a PyTorch NN Classifier

Make sure that pytorch package is [installed](https://pytorch.org/get-started/locally/).

In [ ]:
import torch
from torch.autograd import Variable

print('PyTorch Version: {}'.format(torch.__version__))

#### Step 1. Load data

In this step, we are going to:
1. Load the data to Pandas Dataframe.
1. Convert the class feature (species) from string to a numeric indicator.
1. Split the Dataframe into input feature (xtrain) and target feature (ytrain).

In [ ]:
import pandas as pd

CLASS_VOCAB = ['setosa', 'versicolor', 'virginica']

datatrain = pd.read_csv(LOCAL_DATA_FILE, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

#change string value to numeric
datatrain.loc[datatrain['species']=='Iris-setosa', 'species']=0
datatrain.loc[datatrain['species']=='Iris-versicolor', 'species']=1
datatrain.loc[datatrain['species']=='Iris-virginica', 'species']=2
datatrain = datatrain.apply(pd.to_numeric)

#change dataframe to array
datatrain_array = datatrain.values

#split x and y (feature and target)
xtrain = datatrain_array[:,:4]
ytrain = datatrain_array[:,4]

input_features = xtrain.shape[1]
num_classes = len(CLASS_VOCAB)

print('Records loaded: {}'.format(len(xtrain)))
print('Number of input features: {}'.format(input_features))
print('Number of classes: {}'.format(num_classes))

#### Step 2. Set model parameters
You can try different values for **hidden_units** or **learning_rate**.

In [ ]:
HIDDEN_UNITS = 10
LEARNING_RATE = 0.1

#### Step 3. Define the PyTorch NN model
Here, we build a a neural network with one hidden layer, and a Softmax output layer for classification.

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(input_features, HIDDEN_UNITS),
    torch.nn.Sigmoid(),
    torch.nn.Linear(HIDDEN_UNITS, num_classes),
    torch.nn.Softmax()
)

loss_metric = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=LEARNING_RATE)

#### Step 4. Train the model
We are going to train the model for **num_epoch** epochs.

In [ ]:
NUM_EPOCHS = 10000

for epoch in range(NUM_EPOCHS):
    
    x = Variable(torch.Tensor(xtrain).float())
    y = Variable(torch.Tensor(ytrain).long())
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_metric(y_pred, y)
    loss.backward()
    optimizer.step()
    if (epoch) % 1000 == 0:
        print('Epoch [{}/{}] Loss: {}'.format(epoch+1, NUM_EPOCHS, round(loss.item(),3)))
        
print('Epoch [{}/{}] Loss: {}'.format(epoch+1, NUM_EPOCHS, round(loss.item(),3)))

#### Step 5. Save the model

Triton inference server requires PyTorch models to be saved as [the TorchScript format](https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html).

In [ ]:
# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 4)

In [ ]:
MODEL_NAME = "pytorch"
LOCAL_MODEL_DIRECTORY = f"{MODEL_ARTIFACTS_DIRECTORY}/{MODEL_NAME}"
LOCAL_MODEL_VERSION_1 = f"{LOCAL_MODEL_DIRECTORY}/1"

%mkdir $LOCAL_MODEL_DIRECTORY
%mkdir $LOCAL_MODEL_VERSION_1

In [ ]:
LOCAL_MODEL_FILE = f"{LOCAL_MODEL_VERSION_1}/model.pt"

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

# Save the TorchScript model
traced_script_module.save(LOCAL_MODEL_FILE)

#### Step 6. Test the loaded TorchScript model for predictions

In [ ]:
iris_classifier = torch.jit.load(LOCAL_MODEL_FILE)

def predict_class(instances):
    instances = torch.Tensor(instances)
    output = iris_classifier(instances)
    _ , predicted = torch.max(output, 1)
    return predicted

predicted = predict_class(xtrain[0:5])
print([CLASS_VOCAB[class_index] for class_index in predicted])

## Prepare for model configuration for Triton

Each model in a [model repository](https://github.com/triton-inference-server/server/blob/main/docs/model_repository.md) must include a [model configuration](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md) that provides required and optional information about the model. To load PyTorch models, the model configuration needs to set `backend` and `platform` to `pytorch` and `pytorch_libtorch` respectively. There are also [special conventions for PyTorch backend](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md).

In [ ]:
%%writefile $LOCAL_MODEL_DIRECTORY/config.pbtxt
backend: "pytorch"
platform: "pytorch_libtorch"
max_batch_size: 8
input {
    name: "INPUT__0"
    data_type: TYPE_FP32
    dims: [ 4 ]
}
output {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [ 1 ]
}

## Upload model artifacts to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.pt` (model artifact)
* `config.pbtxt` (model configuration)

Run the following commands to upload your files:

In [ ]:
!gsutil cp -r {MODEL_ARTIFACTS_DIRECTORY}/* {BUCKET_NAME}/{MODEL_ARTIFACT_DIR}/
!gsutil ls {BUCKET_NAME}/{MODEL_ARTIFACT_DIR}/

## Use NVIDIA Triton inference server

### Initiate Local Model with NVIDIA Triton inference server

In this tutorial, we use [NVIDIA Triton inference server](https://developer.nvidia.com/nvidia-triton-inference-server) to serve the PyTorch model. The images are hosted on `nvcr.io` and all released tags can be found in [this link](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver/tags).

In [ ]:
TRITON_VERSION = "21.08"

With the Custom Prediction Routine features, Vertex SDK allows you to load the existing images to test them locally and then deploy them to Vertex AI Prediction easily.

To use Triton inference server on Vertex AI, we need to set up needed parameters such as predict route, health route, ports, and args. By default, Triton listens for HTTP requests on port `8000`.

In [ ]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel.create(
    serving_container_image_uri=f"nvcr.io/nvidia/tritonserver:{TRITON_VERSION}-py3",
    serving_container_predict_route=f"/v2/models/{MODEL_NAME}/infer",
    serving_container_health_route=f"/v2/models/{MODEL_NAME}",
    serving_container_ports=[8000],
    serving_container_args=["tritonserver", "--model-repository=$(AIP_STORAGE_URI)"],
)

You can check out the serving container spec of the image.

In [ ]:
local_model.get_serving_container_spec()

### Store test instances

To learn more about formatting input instances in JSON, [read the documentation.](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
INPUT_FILE = "instances.json"

In [ ]:
%%writefile $INPUT_FILE
{
    "id": "0",
    "inputs": [
        {
            "name": "INPUT__0",
            "shape": [2, 4],
            "datatype": "FP32",
            "data": [[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]]
        }
    ]
}

### Run and test the container locally

#### Set up credentials

Setting up credentials is only required to run the custom serving container locally. Credentials set up is required to execute the `Predictor`'s `load` function, which downloads the model artifacts from Google Cloud Storage.

To access Google Cloud Storage in your project, you'll need to set up credentials by using one of the following:

1. User account

1. Service account

You can learn more about each of the above [here](https://cloud.google.com/docs/authentication#principals).

**Option 1. Use Google user credentials**

First authorize Google auth library to access the Cloud Platform with Google user credentials. This step involves an interactive prompt which requires user inputs. If you are using a non-interactive shell, you should open a terminal to run this command. See [here](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html) for how to open a terminal in a Vertex Workbench notebook environment.

Note that if you are running the command below on a machine without a window manager/browser, it will prompt you to run the command `gcloud auth application-default login --remote-bootstrap="..."`. Only machines that can launch web browsers, e.g. laptops or workstations, are allowed to run this command. After completing the authentication steps using your browser, you'll need to copy and paste the verification code back into the original terminal to continue the login flow. If you are running this command on a machine with web browser support, there is no need to use a separate machine.

This command will print the credential file as `Credentials saved to file: [CREDENTIALS_FILE]`.

In [ ]:
!gcloud auth application-default login

Specify the credential file, which is a json file, as the path.

In [ ]:
CREDENTIALS_FILE = "[CREDENTIALS_FILE]"  # @param {type:"string"}

Next, authorize gcloud to access the Cloud Platform with Google user credentials. The user credentials need to have `setIamPolicy` permission in the project. See [here](https://cloud.google.com/resource-manager/docs/access-control-proj) for more details.

Similar to the previous step, this also requires user inputs. Instructions for the previous step apply here as well.

In [ ]:
!gcloud auth login

Grant roles to the user account. Use the email address that you used in the gcloud auth step, e.g., `myemail@gmail.com`. Since we will be using this user account to download the model artifacts from Google Cloud Storage, we grant the user account **Storage Object Viewer** role. See [IAM roles for Cloud Storage](https://cloud.google.com/storage/docs/access-control/iam-roles) for more details.

In [ ]:
USER_ACCOUNT = "[USER_ACCOUNT]"  # @param {type:"string"}

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=user:$USER_ACCOUNT --role=roles/storage.objectViewer

Initialize the Vertex AI SDK with the project ID, which is required for using user credentials.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

**Option 2. Use Google Service Account credentials**

Skip this if you've already completed Option 1 above. First enable the IAM API if it's not already enabled.

In [ ]:
!gcloud services enable iam.googleapis.com

Specify service account name which should be unique in your project.

In [ ]:
SERVICE_ACCOUNT = "[SERVICE_ACCOUNT]"  # @param {type:"string"}

Create the service account.

In [ ]:
!gcloud iam service-accounts create $SERVICE_ACCOUNT

Authorize gcloud to access the Cloud Platform with Google user credentials. The user credentials need to have `setIamPolicy` permission in the project. See [here](https://cloud.google.com/resource-manager/docs/access-control-proj) for more details.

This step involves an interactive prompt which requires user inputs. If you are using a non-interactive shell, you should open a terminal to run this command. See [here](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html) for how to open a terminal in a Vertex Workbench notebook environment.

Note that if you are running the command below on a machine without a window manager/browser, it will prompt you to run the command `gcloud auth application-default login --remote-bootstrap="..."`. Only machines that can launch web browsers, e.g. laptops or workstations, are allowed to run this command. After completing the authentication steps using your browser, you'll need to copy and paste the verification code back into the original terminal to continue the login flow. If you are running this command on a machine with web browser support, there is no need to use a separate machine.

In [ ]:
!gcloud auth login

Grant roles to the service account. Since we will be using this service account to download the model artifacts from Google Cloud Storage, we grant the service account **Storage Object Viewer** role. See [IAM roles for Cloud Storage](https://cloud.google.com/storage/docs/access-control/iam-roles) for more details.

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com \
    --role=roles/storage.objectViewer

Specify the service account key file name.

In [ ]:
CREDENTIALS_FILE = "./credentials.json"

Generate the service account key file. This will be used while running the custom serving container locally.

In [ ]:
!gcloud iam service-accounts keys create $CREDENTIALS_FILE \
    --iam-account="{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com"

#### Run and send requests to the container locally

With the Custom Prediction Routine feature, it is easy to test the container locally.

Note: You need to have the credentials set up in the previous step and pass the path to the credentials while running the container. The service account should have the **Storage Object Viewer** permission.

In this example, the container executes a prediction request and a health check.

Note: It will be a bit slow for the first time you run the following command because we need to pull the image from `nvcr.io`.

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_NAME}/{MODEL_ARTIFACT_DIR}",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )
    
    health_check_response = local_endpoint.run_health_check()

Print out the predict response and its content.

In [ ]:
predict_response, predict_response.content

Print out the health check response and its content.

In [ ]:
health_check_response, health_check_response.content

Also print out all the container logs.

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### Change image uris to Artifact Registry

Because Vertex AI Prediction currently does not support `nvcr.io` repositories, we need to copy the images to Artifact Registry. We can easily initiate another Local Model instance with the copied images.

In [ ]:
local_model_ar = local_model.copy_image(f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}")

Print out the serving container spec of the new Local Model instance. The image uri should be changed to Artifact Registry.

In [ ]:
local_model_ar.get_serving_container_spec()

### Push the container to artifact registry

Configure Docker to access Artifact Registry. Then push your container image to your Artifact Registry repository.

In [ ]:
!gcloud services list

If `artifactregistry.googleapis.com` is not enabled in your project, enable the API before proceeding.

In [ ]:
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
!gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

Use SDK to push the image.

In [ ]:
local_model_ar.push_image()

## Deploy to Vertex AI

### Upload the NVIDIA Triton inference server model

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

Use the LocalModel instance to upload the model. It will populate the container spec automatically for you.

In [ ]:
model = local_model_ar.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_NAME}/{MODEL_ARTIFACT_DIR}",
)

### Deploy the model on Vertex AI
After this step completes, the model is deployed and ready for online prediction.

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

## Send predictions

Need to use [raw predict](https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/raw-predict) to send arbitrary payloads.

### Using REST

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
--data-binary @$INPUT_FILE \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:rawPredict

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}